Midterm Exams

In [1578]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [1579]:
df_train=pd.read_csv('train.csv')

test_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/test.csv?raw=true'
dt=pd.read_csv(test_url)

sample_submission_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/sample_submission.csv?raw=true'

sf=pd.read_csv(sample_submission_url)

In [1580]:
df_train.shape

(188533, 13)

In [1581]:
drop_col=['id','ext_col', 'int_col']
df_train.drop(drop_col,inplace=True,axis=1)
object_columns = df_train.select_dtypes(include=['object']).columns
df_train[object_columns]

,brand,model,fuel_type,engine,transmission,accident,clean_title
0,MINI,Cooper S Base,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,None reported,Yes
1,Lincoln,LS V8,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,At least 1 accident or damage reported,Yes
2,Chevrolet,Silverado 2500 LT,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,None reported,Yes
3,Genesis,G90 5.0 Ultimate,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,None reported,Yes
4,Mercedes-Benz,Metris Base,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,None reported,Yes
...,...,...,...,...,...,...,...
188528,Cadillac,Escalade ESV Platinum,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,None reported,Yes
188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,At least 1 accident or damage reported,Yes
188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,None reported,Yes
188531,Audi,S5 3.0T Prestige,Gasoline,3.0L,1-Speed Automatic,None reported,NaN


In [1582]:
for column in object_columns:
    unique_values = df_train[column].unique()
    print(f"Unique values in '{column}':")
    print(unique_values)
    print()

Unique values in 'brand':
['MINI' 'Lincoln' 'Chevrolet' 'Genesis' 'Mercedes-Benz' 'Audi' 'Ford'
 'BMW' 'Tesla' 'Cadillac' 'Land' 'GMC' 'Toyota' 'Hyundai' 'Volvo'
 'Volkswagen' 'Buick' 'Rivian' 'RAM' 'Hummer' 'Alfa' 'INFINITI' 'Jeep'
 'Porsche' 'McLaren' 'Honda' 'Lexus' 'Dodge' 'Nissan' 'Jaguar' 'Acura'
 'Kia' 'Mitsubishi' 'Rolls-Royce' 'Maserati' 'Pontiac' 'Saturn' 'Bentley'
 'Mazda' 'Subaru' 'Ferrari' 'Aston' 'Lamborghini' 'Chrysler' 'Lucid'
 'Lotus' 'Scion' 'smart' 'Karma' 'Plymouth' 'Suzuki' 'FIAT' 'Saab'
 'Bugatti' 'Mercury' 'Polestar' 'Maybach']

Unique values in 'model':
['Cooper S Base' 'LS V8' 'Silverado 2500 LT' ... 'e-Golf SE'
 'Integra w/A-Spec Tech Package' 'IONIQ Plug-In Hybrid SEL']

Unique values in 'fuel_type':
['Gasoline' 'E85 Flex Fuel' nan 'Hybrid' 'Diesel' 'Plug-In Hybrid' '–'
 'not supported']

Unique values in 'engine':
['172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel'
 '252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel'
 '320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capabil

In [1583]:
from datetime import datetime

current_year = datetime.now().year
car_brands = ['MINI', 'Lincoln', 'Chevrolet', 'Genesis', 'Mercedes-Benz', 'Audi', 'Ford', 'BMW', 'Cadillac', 'Land',
              'GMC', 'Toyota', 'Hyundai', 'Volvo', 'Volkswagen', 'Buick', 'Rivian', 'Hummer', 'Alfa', 'INFINITI',
              'Jeep', 'Porsche', 'Honda', 'Lexus', 'Dodge', 'Nissan', 'Jaguar', 'Kia', 'Mitsubishi', 'Rolls-Royce',
              'Maserati', 'Pontiac', 'Saturn', 'Bentley', 'Tesla', 'Mazda', 'Subaru', 'Ferrari', 'Aston', 'Acura',
              'Lamborghini', 'Chrysler', 'RAM', 'McLaren', 'Lucid', 'Lotus', 'Scion', 'Plymouth', 'Suzuki', 'FIAT',
              'Saab', 'Bugatti', 'Mercury', 'Karma', 'Maybach', 'Polestar', 'smart']

luxury_cars = ['Lincoln', 'Genesis', 'Mercedes-Benz', 'Audi', 'BMW', 'Cadillac', 'Volvo', 'Porsche', 'Lexus',
               'Jaguar', 'Rolls-Royce', 'Maserati', 'Bentley', 'Tesla', 'Ferrari', 'Aston', 'Acura', 'Lamborghini',
               'McLaren', 'Lucid', 'Lotus', 'Bugatti', 'Karma', 'Maybach', 'Polestar']

transmissions = ['A/T', 'Transmission w/Dual Shift Mode', '7-Speed A/T', '8-Speed A/T', '10-Speed Automatic',
                 '1-Speed A/T', '6-Speed A/T', '10-Speed A/T', '9-Speed A/T', '8-Speed Automatic', '9-Speed Automatic',
                 '5-Speed A/T', 'Automatic', '7-Speed Automatic with Auto-Shift', 'CVT Transmission', '5-Speed M/T',
                 'M/T', '6-Speed M/T', '6-Speed Automatic', '4-Speed Automatic', '7-Speed M/T', '2-Speed A/T',
                 '1-Speed Automatic', 'Automatic CVT', '4-Speed A/T', '6-Speed Manual', 'Transmission Overdrive Switch',
                 '8-Speed Automatic with Auto-Shift', '7-Speed Manual', '7-Speed Automatic', '9-Speed Automatic with Auto-Shift',
                 '6-Speed Automatic with Auto-Shift', '6-Speed Electronically Controlled Automatic with O', 'F', 'CVT-F',
                 '8-Speed Manual', 'Manual', '–', '2', '6 Speed At/Mt', '5-Speed Automatic', '2-Speed Automatic',
                 '8-SPEED A/T', '7-Speed', 'Variable', 'Single-Speed Fixed Gear', '8-SPEED AT', '10-Speed Automatic with Overdrive',
                 '7-Speed DCT Automatic', 'SCHEDULED FOR OR IN PRODUCTION', '6-Speed', '6 Speed Mt']

automatic_transmissions = ['A/T', 'Transmission w/Dual Shift Mode', '7-Speed A/T', '8-Speed A/T', '10-Speed Automatic',
                           '1-Speed A/T', '6-Speed A/T', '10-Speed A/T', '9-Speed A/T', '8-Speed Automatic', '9-Speed Automatic',
                           '5-Speed A/T', 'Automatic', '7-Speed Automatic with Auto-Shift', 'CVT Transmission', '6-Speed Automatic',
                           '4-Speed Automatic', '2-Speed A/T', '1-Speed Automatic', 'Automatic CVT', '4-Speed A/T',
                           '8-Speed Automatic with Auto-Shift', '7-Speed Automatic', '9-Speed Automatic with Auto-Shift',
                           '6-Speed Automatic with Auto-Shift', '6-Speed Electronically Controlled Automatic with O', 'F', 'CVT-F',
                           '5-Speed Automatic', '2-Speed Automatic', '8-SPEED A/T', '7-Speed', 'Variable', 'Single-Speed Fixed Gear',
                           '8-SPEED AT', '10-Speed Automatic with Overdrive', '7-Speed DCT Automatic']

fuel_types = ['Gasoline', 'E85 Flex Fuel', np.nan, 'Hybrid', 'Diesel', 'Plug-In Hybrid', '–', 'not supported']

fuel_dict = {
    'Gasoline': 1,
    'E85 Flex Fuel': 3,
    'Hybrid': 5,
    'Diesel': 2,
    'Plug-In Hybrid': 4,
    '–': 0,
    'not supported': 0,
    np.nan: 0
}


clean_title_dict = {
    'Yes': 1,
    np.nan: 0
}


engines = [
    '172.0 HP 1.6L 4 Cylinder Engine Gasoline Fuel',
    '252.0 HP 3.9L 8 Cylinder Engine Gasoline Fuel',
    '320.0 HP 5.3L 8 Cylinder Engine Flex Fuel Capability',
    '78.0 HP 1.2L 3 Cylinder Engine Gasoline Fuel',
    '139.0 HP 1.6L 4 Cylinder Engine Plug-In Electric/Gas',
    '313.0 HP 2.0L 4 Cylinder Engine Plug-In Electric/Gas',
    'Electric Engine',
    'Hybrid Engine'
]

engine_dict = {
    '172.0 HP 1.6L 4 Cylinder Engine Gasoline Fuel': 2,
    '252.0 HP 3.9L 8 Cylinder Engine Gasoline Fuel': 3,
    '320.0 HP 5.3L 8 Cylinder Engine Flex Fuel Capability': 4,
    '78.0 HP 1.2L 3 Cylinder Engine Gasoline Fuel': 1,
    '139.0 HP 1.6L 4 Cylinder Engine Plug-In Electric/Gas': 5,
    '313.0 HP 2.0L 4 Cylinder Engine Plug-In Electric/Gas': 6,
    'Electric': 7,
    'Hybrid': 8
}

df = pd.DataFrame({'Engine': [
    '172.0 HP 1.6L 4 Cylinder Engine Gasoline Fuel',
    '252.0 HP 3.9L 8 Cylinder Engine Gasoline Fuel',
    '320.0 HP 5.3L 8 Cylinder Engine Flex Fuel Capability',
    '78.0 HP 1.2L 3 Cylinder Engine Gasoline Fuel',
    '139.0 HP 1.6L 4 Cylinder Engine Plug-In Electric/Gas',
    '313.0 HP 2.0L 4 Cylinder Engine Plug-In Electric/Gas',
    'Electric',
    'Hybrid',
    np.nan
]})


transmission_dict = {trans: 1 if trans in automatic_transmissions else 0 for trans in transmissions}
car_dict = {brand: 1 if brand in luxury_cars else 0 for brand in car_brands}
clean_title_values = ['Yes', np.nan]


df_train['engineRank'] = df_train['engine'].map(engine_dict).fillna(0).astype(int)
df_train['cleanTitleValue'] = df_train['clean_title'].map(clean_title_dict).astype(int)
df_train['luxury_brand'] = df_train['brand'].map(car_dict).astype(int)
df_train['fuelValue'] = df_train['fuel_type'].map(fuel_dict).astype(int)
df_train['isAutomatic'] = df_train['transmission'].map(transmission_dict).astype(int)
df_train['model_year'] = df_train['model_year'].astype(int)
df_train['car_age'] = current_year - df_train['model_year']

dt['engineRank'] = dt['engine'].map(engine_dict).fillna(0).astype(int)
dt['cleanTitleValue'] = dt['clean_title'].map(clean_title_dict).astype(int)
dt['luxury_brand'] = dt['brand'].map(car_dict).astype(int)
dt['fuelValue'] = dt['fuel_type'].map(fuel_dict).astype(int)
dt['isAutomatic'] = dt['transmission'].map(transmission_dict).astype(int)
dt['model_year'] = dt['model_year'].astype(int)
dt['car_age'] = current_year - dt['model_year']

In [1584]:
from sklearn.model_selection import train_test_split

x = df_train.drop(['engineRank','fuelValue','luxury_brand','price','brand','model','fuel_type','engine','transmission','accident','clean_title','model_year'], axis = 1)
y = df_train['price']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [1585]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   brand            188533 non-null  object
 1   model            188533 non-null  object
 2   model_year       188533 non-null  int64 
 3   milage           188533 non-null  int64 
 4   fuel_type        183450 non-null  object
 5   engine           188533 non-null  object
 6   transmission     188533 non-null  object
 7   accident         186081 non-null  object
 8   clean_title      167114 non-null  object
 9   price            188533 non-null  int64 
 10  engineRank       188533 non-null  int64 
 11  cleanTitleValue  188533 non-null  int64 
 12  luxury_brand     188533 non-null  int64 
 13  fuelValue        188533 non-null  int64 
 14  isAutomatic      188533 non-null  int64 
 15  car_age          188533 non-null  int64 
dtypes: int64(9), object(7)
memory usage: 23.0+ MB


In [1586]:
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

LinearRegression()

In [1587]:
y_pred_train = linear_reg.predict(X_train)
y_pred_test = linear_reg.predict(X_test)

In [1588]:
train_mse = mean_squared_error(y_train, y_pred_train, squared=False)
test_mse = mean_squared_error(y_test, y_pred_test, squared=False)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print(f"Training MSE: {train_mse}")
print(f"Testing MSE: {test_mse}")
print(f"Training R2 Score: {train_r2}")
print(f"Testing R2 Score: {test_r2}")

Training MSE: 78023.58754198528
Testing MSE: 69094.8026263072
Training R2 Score: 0.07926414270523541
Testing R2 Score: 0.09595694308652902


In [1589]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   brand            188533 non-null  object
 1   model            188533 non-null  object
 2   model_year       188533 non-null  int64 
 3   milage           188533 non-null  int64 
 4   fuel_type        183450 non-null  object
 5   engine           188533 non-null  object
 6   transmission     188533 non-null  object
 7   accident         186081 non-null  object
 8   clean_title      167114 non-null  object
 9   price            188533 non-null  int64 
 10  engineRank       188533 non-null  int64 
 11  cleanTitleValue  188533 non-null  int64 
 12  luxury_brand     188533 non-null  int64 
 13  fuelValue        188533 non-null  int64 
 14  isAutomatic      188533 non-null  int64 
 15  car_age          188533 non-null  int64 
dtypes: int64(9), object(7)
memory usage: 23.0+ MB


In [1590]:
dt.drop(['fuelValue','luxury_brand','engineRank','ext_col','id','int_col','brand','model','fuel_type','engine','transmission','accident','clean_title','model_year'],inplace=True,axis=1)

In [1591]:
dt.info()
dt.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype
---  ------           --------------   -----
 0   milage           125690 non-null  int64
 1   cleanTitleValue  125690 non-null  int64
 2   isAutomatic      125690 non-null  int64
 3   car_age          125690 non-null  int64
dtypes: int64(4)
memory usage: 3.8 MB


(125690, 4)

In [1592]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

69094.8026263072

In [1593]:
id = sf.pop('id')
y_pred = model.predict(dt)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': id,
    'price': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_file.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
